In [2]:
import pandas as pd
from docplex.mp.model import Model

# Load data
df_demand = pd.read_csv("demand.csv").set_index('County')
df_dist = pd.read_csv("distance_km.csv")

# Parameters
Counties = df_demand.index.tolist()
Demand = df_demand['Demand'].to_dict()
Capacity = 10000
Range = 150

# Initialize CanServe dictionary
CanServe = {(i, i): 1 for i in Counties}
for i in df_dist.index:
    if df_dist['Distance'][i] <= Range:
        c1 = df_dist['County1'][i]
        c2 = df_dist['County2'][i]
        CanServe[c1, c2] = 1
        if c1 not in Counties:
            print(f"ERROR: {c1} not in Counties")
        if c2 not in Counties:
            print(f"ERROR: {c2} not in Counties")

# Create model
mdl = Model()

# Decision variables
x = mdl.binary_var_dict(Counties, name='x')
y = mdl.continuous_var_dict(CanServe.keys(), name='y')

# Objective: minimize number of base stations
mdl.minimize(mdl.sum(x[j] for j in Counties))

# Constraints
# Capacity constraint
for j in Counties:
    mdl.add_constraint(mdl.sum(Demand[i] * y[i, j] for i in Counties if (i, j) in CanServe) <= Capacity * x[j])

# Demand fulfillment constraint
for i in Counties:
    mdl.add_constraint(mdl.sum(y[i, j] for j in Counties if (i, j) in CanServe) == 1)

# Distance constraint (implicitly handled by CanServe dictionary)

# Optimize model
mdl.solve()

# Output results
selected_base_stations = [j for j in Counties if x[j].solution_value > 0.5]
print(f'Number of base stations needed: {len(selected_base_stations)}')
print(f'Selected base stations: {selected_base_stations}')
delivery_plan = {(i, j): y[i, j].solution_value for i in Counties for j in Counties if (i, j) in CanServe and y[i, j].solution_value > 0.001}
print(f'Delivery plan: {delivery_plan}')


Number of base stations needed: 11
Selected base stations: ['Berks', 'Bradford', 'Bucks', 'Cameron', 'Clearfield', 'Columbia', 'Elk', 'Northampton', 'Snyder', 'Somerset', 'York']
Delivery plan: {('Adams', 'Berks'): 1.0, ('Allegheny', 'Clearfield'): 0.6984796468857283, ('Allegheny', 'Somerset'): 0.3015203531142717, ('Armstrong', 'Cameron'): 1.0, ('Beaver', 'Somerset'): 1.0, ('Bedford', 'Somerset'): 1.0, ('Berks', 'York'): 1.0, ('Blair', 'Cameron'): 1.0, ('Bradford', 'Cameron'): 1.0, ('Bucks', 'Bucks'): 0.3601995778161582, ('Bucks', 'Columbia'): 0.6398004221838418, ('Butler', 'Elk'): 1.0, ('Cambria', 'Cameron'): 1.0, ('Cameron', 'Cameron'): 1.0, ('Carbon', 'Bradford'): 1.0, ('Centre', 'Bradford'): 0.11292834890965664, ('Centre', 'Clearfield'): 0.4104361370716516, ('Centre', 'Elk'): 0.47663551401869175, ('Chester', 'Northampton'): 0.12025012025012027, ('Chester', 'York'): 0.8797498797498797, ('Clarion', 'Cameron'): 1.0, ('Clearfield', 'Cameron'): 1.0, ('Clinton', 'Snyder'): 1.0, ('Columbi